# Introduction to regularized regression

Despite ordinary least squares (OLS) being the best unbiased linear estimator for a system of equations, biased (regularized) estimators can achieve lower mean-squared errors, especially on unseen or held out data. This is particularly true when the number of predictors is high or compared to the number of observations (which can often be true for large neural encoding models)! OLS also has no unique solution when there are more predictors than samples (this is referred to as an underdetermined system), therefore it will cross-validate quite poorly in this regime. 

Regularization will help overcome this problem because it reduces variance in order to improve model predictions on held out data. Regularization is also particularly useful when there is multicollinearity between your regressors. OLS suffers from large standard errors on the $\beta$ estimates in this regime.


When building encoding models for neural data, regressors often may be collinear. Let's take a basic neuroscience example of a mouse that must press a lever to recieve a food reward. If a food reward is delivered every time the lever is pressed, it will be quite difficult to separate out the neural encoding of action (lever pressing) from the encoding of reward (recieving food) because these variables always occur together (and occur closely in time). When designing behavioral tasks, think how you might avoid collinearities. How might we modify this task so we can better isolate the encoding of action and reward?

### Ordinary least squares (OLS), ridge, and lasso regression
The goal of regression is to minimize the sum of squared errors (SSE). Below are shown the different SSE formulations for OLS, ridge, and lasso regression.
$$
SSE_{OLS} = \sum_{i=1}^{n}{(y_i - \hat{y}_i)^2}
$$
$$
SSE_{Ridge} = \sum_{i=1}^{n}{(y_i - \hat{y}_i)^2} + \alpha\sum_{j=1}^{P}{\beta_j^2}
$$
$$
SSE_{Lasso} = \sum_{i=1}^{n}{(y_i - \hat{y}_i)^2} + \lambda\sum_{j=1}^{P}{|\beta_j|}
$$
You can see that the ridge and lasso equations include an additional term that penalizes large $\beta$ values. The only difference between ridge and lasso is that ridge is squared and lasso takes the absolute value of the weights instead. In practice, this means that lasso will have sparse solutions (less informative predictors will have $\beta$ weights set to 0) while ridge regression will spread out variance across predictors, even if they are correlated. 

By minimizing the above functions, we can compute the OLS and Ridge estimators! We will skip over the derivation of the closed form solutions, but just know that they come from minimizing the above equations. 

$$
\hat{\beta}_{OLS} = (X^TX)^{-1}X^Ty
$$
$$
\hat{\beta}_{Ridge} = (X^TX + \alpha I)^{-1}X^Ty
$$
Lasso regression has no closed form solution. It must be solved with other techniques such as gradient descent, so we will omit it for now. 

For the rest of this notebok we are going to implement a lot of things "by hand" to get an intuition of how the regression is done under the hood. In the next notebook we will make use of Python libraries (mostly sklearn) to do a lot of the heavy lifting for us.


In [ ]:
#First import the standard toolboxes
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

We will import a dataset that gives the average miles per gallon (MPG) for many different cars. The displacement, horsepower, cylinders, and other information about the cars is also included. We will try to see how well we can predict fuel efficiency from these variables.

In [ ]:
PREDICTOR_LABELS = ['cylinders','displacement','horsepower','weight','acceleration','model_year','american','japanese']
dataset = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/mpg.csv')
dataset.head()

In [ ]:
# Preprocess the data
dataset = dataset.dropna()
y = dataset.mpg.values
x = dataset[PREDICTOR_LABELS[:-2]].to_numpy()

is_american = (dataset.origin == 'usa').values #dummy variable
is_japanese = (dataset.origin == 'japan').values #dummy variable

x = np.vstack((x.T, is_american, is_japanese)).T
print(f'The shape of x is {x.shape}')
print(f'The shape of y is {y.shape}')

# Plot each of the regressors vs MPG so we have some idea how they relate
fig,axs = plt.subplots(4,2,figsize=(6,8),dpi=100)
for i,ax in enumerate(np.ravel(axs)):
    ax.scatter(x[:,i],y, s=3)
    ax.set_xlabel(PREDICTOR_LABELS[i])
    ax.set_ylabel('Fuel Efficiency (MPG)')
fig.tight_layout()

print('\nGiven the plots below, what do you expect the regression weights to look like for each variable?')

In [ ]:
# More data preprocessing
x = #TODO: normalize the columns of x. Why must we do this for ridge regression?
y = #TODO: normalize the columns of y
y = #TODO: mean center y. Why must we do this for ridge regression?

### Excercise 1: Implement OLS and Ridge regression
Use the equations at the beginning of the notebook to compute the $\beta$ estimates using OLS and ridge regression. 

In [ ]:
def ols_solver(x,y):
    """
    TODO: use the equation at the beginning of the notebook to compute the OLS solution for beta as a function of x and y. 
    Hint: the @ symbol is equivalent to np.matmul()
    """
    return betas

def ridge_solver(x,y,alpha):
    """
    TODO: use the equation at the beginning of the notebook to compute the ridge for beta as a function of x, y, and alpha. 
    Hint: the @ symbol is equivalent to np.matmul()
    """
    return betas

RIDGE_ALPHA =  #TODO: What alpha value will make ridge regression equal to ordinary least squares?

ols_betas = ols_solver(x,y)
ridge_betas = ridge_solver(x,y,RIDGE_ALPHA)

for label, ols_beta, ridge_beta in zip(PREDICTOR_LABELS, ols_betas, ridge_betas):
    print(f'OLS beta weight for {label}: {ols_beta}')
    #print(f'ridge beta weight for {label}: {ridge_beta}')

assert np.array_equiv(ols_betas,ridge_betas), "Betas not equal, change RIDGE_ALPHA so that they will become equal"
print('\nHow do the beta weights match up with what you expected from the plots above?')


### Excercise 2: Generate model predictions and implement a scoring function
There are many ways to assess a models performance (many of them are also built into scikit-learn, which we will use later). But for now, lets practice building our own custom scoring function. We will use the *coefficient of determination* ($R^2$), which is a useful scoring metric to evaluate many different models.
$$
R^2 = 1 - \frac{\sum_i(y_i - \hat{y}_i)^2}{\sum_i(y_i - \bar{y})^2}
$$

This is the proportion of the dependent variable that is predictable from the independent variables. Usually, its value ranges from 0 to 1, though values can go negative when scoring on held out (cross-validated) data.

What might a negative $R^2$ mean?

In [ ]:
def r_squared_score(y_true, y_pred):
    """
    TODO: using the above equation, implement a function that returns 
    the coefficient of determination (R^2) as a function of the true y
    values and the model predictions of y (yhat in the equation above)
    """
    return r_squared

y_pred_ols = #TODO: generate OLS predictions using the previously calculated beta weights
y_pred_ridge = #TODO: generate ridge predictions using the previously calculated beta weights

r2_ols = r_squared_score(y,y_pred_ols)
r2_ridge = r_squared_score(y,y_pred_ridge) # if the ridge parameter (alpha) was set to zero, these scores should be equivalent

print(f'OLS R-squared: {r2_ols}')
print(f'Ridge R-squared: {r2_ridge}')

Now let's vary the ridge parameter and see how our model performs. Do you think ridge or OLS will perform better? Where will they be equivalent? Remember, we are not doing cross-validation yet.

In [ ]:
ALPHAS = 10**np.linspace(-15,7, 100)

scores = np.empty_like(ALPHAS) # empty array the same shape as ALPHAS

"""
TODO: Compute the R^2 for each value in ALPHAS and add that to the numpy array called "scores".
Hint: a for loop may be helpful or you can use map() to map a function.
"""

ols_score = #TODO: compute the ols R^2 for reference

# plotting
plt.scatter(ALPHAS,scores, s=10, color='black', label='ridge')
plt.scatter(ALPHAS[0], ols_score, s=5, color='red',label='ordinary least squares') # plot OLS score for reference
plt.hlines(0,ALPHAS[0],ALPHAS[-1],linestyle='--',linewidth=.5, color='black')
plt.legend()
plt.xlabel('Alpha')
plt.xscale('log')
plt.ylabel('$R^2$ (training set)')
plt.show()

### Excercise 3: Implement cross-validation

As you saw in the previous exercise, ridge regression will actually never perform better than ordinary least squares when assessing performance on training data. The power of regularization comes when generating predictions on held out data (cross-validation). When assessing model performance, it is best practice to do this on cross validated data. This ensures our model is not over-fitting and that it will be able to make reliable predictions on new data. In this toy example about fuel efficiency, we want a model that will give us a good prediction for a *new* car where the fuel efficiency is not availible. 

Here, we are going to implement k-fold cross validation. K-fold cross validation splits our data sets into k chunks. We will leave one chunk of the data out for testing (with our function `r_squared_score()`). We will train our model on the remaining data, and then go back and make predictions on the held out data.

Here's a nice visualization of 5-fold crossvalidation

![k-fold crossval figure](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*AAwIlHM8TpAVe4l2FihNUQ.png)

https://towardsdatascience.com/cross-validation-k-fold-vs-monte-carlo-e54df2fc179b

SKLearn has several useful functions for cross-validation. We will make use of one of them here.

In [ ]:
from sklearn.model_selection import KFold
N_FOLDS = 5
ALPHAS = 10**np.linspace(-15,7, 100) #try some different alpha values

def cross_val_score(x, y, n_folds, scoring_function, fitting_function, **fitting_func_kwargs):
    """
    This function takes x and y and will return a cross-validated score with "n_folds" folds
    It accepts an arbitrary scoring and fitting function and so it can be used with a vaiety of models
    and scoring metrics.
    """

    kf = #TODO: create an sklearn.model_selection.KFold object with "n_folds" folds. Call it "kf". Can you make it randomly split up the data with the "shuffle" parameter? SKLearn's documentation will be helpful here.

    train_scores = np.empty((n_folds))
    test_scores = np.empty((n_folds))
    for j, (train_index, test_index) in enumerate(kf.split(x)): # iterate over cross-validation folds
        x_train = #TODO: get train data for x
        x_test = #TODO: get test data for x
        y_train = #TODO: get train data for y
        y_test = #TODO: get test data for y

        betas = fitting_function(x=x_train,y=y_train,**fitting_func_kwargs)
        y_train_predictions = x_train @ betas
        y_test_predictions = x_test @ betas
        train_scores[j] = scoring_function(y_train, y_train_predictions)
        test_scores[j] = scoring_function(y_test, y_test_predictions)
    ##############
    """
    TODO: get the average training and testing scores over folds and return those
    """
    avg_train_score = 
    avg_test_score = 
    ##############
    return avg_train_score, avg_test_score

In [ ]:
################################
train_scores, test_scores = [], []
"""
TODO: Use the function we created above and iterate over ALPHAS to get the train_r2 and test_r2 for each alpha.
Then append these values to the train_scores and test_scores lists

Take careful note of the parameters for our cross_val_score() function. When called, it can accept arbitrary functions to fit and score.
Luckily, we have already created these...
"""
################################

ols_train_r2, ols_test_r2 = cross_val_score(x, y, N_FOLDS, r_squared_score, ols_solver) #run OLS for comparison

# plotting
plt.scatter(ALPHAS, train_scores, s=10, color='black',label='ridge train')
plt.scatter(ALPHAS, test_scores, s=10, color='red',label='ridge test')
plt.scatter(ALPHAS[0], ols_train_r2, s=10, color='blue',label='ols train')
plt.scatter(ALPHAS[0], ols_test_r2, s=10, color='orange',label='ols test')
plt.hlines(0,ALPHAS[0],ALPHAS[-1],linestyle='--',linewidth=.5, color='black')
plt.legend()
plt.xlabel('alpha')
plt.xscale('log')
plt.ylabel('$R^2$')

print(f'OLS test performance: {ols_test_r2}')
print(f'Best ridge test performance: {np.max(test_scores)}')

Look at the above plot and consider the following:
- Are the OLS or ridge models overfitting?
- is ridge regression needed in this case?

### Excercise 4: Regression with collinear regressors and underdetermined systems

In the plot above, you may have noticed that ridge regression test performance is only *slightly* better than ordinary least squares. This is because we have a well conditioned dataset. As we introduced at the beginning of the notebook, where regularization becomes most helpful is when you have few observations relative to the number of regressors or when regressors are collinear. Let's manipulate our regressor matrix $X$ and compare OLS and ridge again.

In [ ]:
# augment the matrix x with noisy versions of itself
random_offset = x + np.random.normal(scale=1e-5, size=x.shape) # same as x but with small random differences
random_offset_2 = x + np.random.normal(scale=1e-3, size=x.shape)
random_offset_3 = x + np.random.normal(scale=1e-7, size=x.shape)
x_augmented = np.hstack((x, random_offset, random_offset_2, random_offset_3)) # add columns to x that are nearly identical. This is an "ill-conditioned" matrix but still full rank. 

#x_augmented = np.hstack((x, x)) # This matrix exhibits multicollinearity (it is not full rank). Uncomment and see how OLS vs ridge handle this.
x_augmented = x_augmented / np.max(x_augmented, axis=0) #renormalize

print(f'Shape of x is originally {x.shape}')
print(f'Shape of x with data augmentation is {x_augmented.shape}')

# now let's subsample x and y so that the number of observations is closer to the number of regressors
N_SUBSAMPLE = 50 #The number of observations to randomly grab 
##################
# TODO: Randomly subsample from x_augmented and y, grabbing "N_SUBSAMPLE" observations
# Hint: np.random.choice() may be helpful
x_subsample =
y_subsample =
print(f'Shape of x after subsampling is {x_subsample.shape}')
##################


In [ ]:
# now fit this new dataset (x_subsample and y_subsample)

################################
ALPHAS = 10**np.linspace(-15,7, 100)
train_scores, test_scores = [], []
"""
TODO: Iterate over ALPHAS to get the train_r2 and test_r2 for each alpha.
Then append these values to the train_scores and test_scores lists. You should
be able to copy and paste your answer from earlier.
"""
################################

ols_train_r2, ols_test_r2 = cross_val_score(x_subsample, y_subsample, N_FOLDS, r_squared_score, ols_solver) #run OLS for comparison

# plotting
plt.scatter(ALPHAS, train_scores, s=10, color='black',label='ridge train')
plt.scatter(ALPHAS, test_scores, s=10, color='red',label='ridge test')
plt.scatter(ALPHAS[0], ols_train_r2, s=10, color='blue',label='ols train')
plt.scatter(ALPHAS[0], ols_test_r2, s=10, color='orange',label='ols test')
plt.hlines(0,ALPHAS[0],ALPHAS[-1],linestyle='--',linewidth=.5, color='black')
plt.legend()
plt.xlabel('alpha')
plt.xscale('log')
plt.ylim(-2,1.2)
plt.ylabel('$R^2$')

print(f'OLS test R^2: {ols_test_r2}')
print(f'Best ridge test R^2: {np.max(test_scores)}')

best_ridge_alpha_value = #TODO: determine what alpha value led to the highest cross validated score

print(f'Best ridge parameter (alpha): {best_ridge_alpha_value}')


Look at this plot after adding in more predictor variables and reducing the number of observations and consider the following:
- How are the two models performing? Are they overfitting?
- Previously, the OLS and ridge fits had similar $R^2$ values, now they look wildly different for the test data. Why do you think this is?
- See what happens when you modify `N_SUBSAMPLE`. How does this affect the results?
- Will these methods work on a collinear matrix? Try it and see what happens
- What will the $\beta$ weights look like for OLS and ridge? Will there be a unique solution for OLS and ridge?
- In your own data, how might you check to see if regressors are correlated before doing the regression?

### Up next: Selecting the proper value of alpha and fitting to neural data!
Since $\alpha$ is a model hyperparameter, it is important that the proper value is selected to get the best model performance. Cross-validating with many different alphas (what we did above) is one way to do that, but there are other ways it can be done. See Karabastos 2017, *Communications in Statistics* for another method of $\alpha$ selection. The Karabastos method is the one implemented in the Musall 2019 paper we discussed.

In the next notebook, we will fit an encoding model to neural data and solve it with ridge regression!